## People Centered COS

### Attempt to map string location to Lat Long

#### Analysis Questions
  
- Can we map the string location 1 and location 2 to latitude and longitude values reliably?

### Set Up 

In [10]:
pip install geocoder


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install geopy


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install plotly


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import geocoder

from geopy.geocoders import Nominatim

In [16]:
pd.set_option('display.max_rows', 500)

### Load CDOT Data from 2020 - 2024

In [17]:
import sys  
sys.path.insert(1, '../../')

from convert_old_cdot_format_to_new_format import convert_old_cdot_format_to_new_format

In [121]:
# Load CDOT Data for a list of years

years = [2023, 2024]

cdot_base_path = "/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/"

loaded_data_list = []
for y in years:
    filename = f"CDOTRM_CD_Crash_Listing_-_{y}.xlsx"
    full_path = cdot_base_path + filename
    print(full_path)
    year_cdot_pdf = pd.read_excel(cdot_base_path + filename)
    year_cdot_pdf['Crash Year'] = y
    print(list(year_cdot_pdf.columns)[0:5])
    print(year_cdot_pdf.shape)
    
    if y < 2021:
        # call conversion code 
        year_cdot_pdf = convert_old_cdot_format_to_new_format(year_cdot_pdf)

    loaded_data_list.append(year_cdot_pdf) 
    
cdot_pdf_load = pd.concat(loaded_data_list)

/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2023.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(102166, 111)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2024.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(99980, 111)


### Clean Data

In [122]:
print('Loaded Data Size', cdot_pdf_load.shape)

# Drop duplicates
cdot_pdf_withNans_dedup = cdot_pdf_load.drop_duplicates()
print('Data size after drop duplicates', cdot_pdf_withNans_dedup.shape)

display(cdot_pdf_withNans_dedup.head(5))

Loaded Data Size (202146, 111)
Data size after drop duplicates (202146, 111)


,CUID,System Code,Rd_Number,Rd_Section,City_Street,Crash Date,Crash Time,Agency Id,City,County,...,TU-1 NM Alcohol Suspected,TU-2 NM Alcohol Suspected,TU-1 NM Marijuana Suspected,TU-2 NM Marijuana Suspected,TU-1 NM Other Drugs Suspected,TU-2 NM Other Drugs Suspected,Record Status,Processing Status,Last Updated,Crash Year
0,296027,City Street,0455,VERMO,TIMBE,2023-01-01,00:00:00,FTCPD,FORT COLLINS,LARIMER,...,NaN,NaN,NaN,NaN,NaN,NaN,Coded,Completed,Aug 4 2023 11:35PM,2023
1,299624,City Street,0455,HARMO,TIMBE,2023-01-01,00:03:00,FTCPD,FORT COLLINS,LARIMER,...,NaN,NaN,NaN,NaN,NaN,NaN,Coded,Completed,Apr 6 2023 10:08PM,2023
2,289075,City Street,0320,KALAM,CEDAR,2023-01-01,00:09:00,DPD,DENVER,DENVER,...,NaN,NaN,NaN,NaN,NaN,NaN,Coded,Completed,Feb 8 2023 8:29PM,2023
3,293978,City Street,0055,LINVA,LARED,2023-01-01,00:16:00,AURPD,AURORA,ARAPAHOE,...,NaN,NaN,NaN,NaN,NaN,NaN,Coded,Completed,May 5 2023 8:02PM,2023
4,299717,State Highway,095A,12.906,NaN,2023-01-01,00:34:00,WMPD,WESTMINSTER,JEFFERSON,...,NaN,NaN,NaN,NaN,NaN,NaN,Coded,Completed,Feb 15 2023 1:35PM,2023


In [123]:
print(list(cdot_pdf_withNans_dedup.columns))

['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street', 'Crash Date', 'Crash Time', 'Agency Id', 'City', 'County', 'Latitude', 'Longitude', 'Location 1', 'Link', 'Location 2', 'Location', 'Road Description', 'First HE', 'Second HE', 'Third HE', 'Fourth HE', 'MHE', 'Crash Type', 'Approach Overtaking Turn', 'Wild Animal', 'Number Killed', 'Number Injured', 'Injury 00', 'Injury 01', 'Injury 02', 'Injury 03', 'Injury 04', 'Total Vehicles', 'Secondary Crash', 'Construction Zone', 'School Zone', 'Road Contour Curves', 'Road Contour Grade', 'Road Condition', 'Lighting Conditions', 'Weather Condition', 'Weather Condition 2', 'Lane Position', 'TU-1 Direction', 'TU-2 Direction', 'TU-1 Movement', 'TU-2 Movement', 'TU-1 Type', 'TU-2 Type', 'TU-1 Special Function', 'TU-2 Special Function', 'TU-1 Autonomous Vehicle', 'TU-2 Autonomous Vehicle', 'TU-1 Hit And Run', 'TU-2 Hit And Run', 'TU-1 Speed Limit', 'TU-2 Speed Limit', 'TU-1 Estimated Speed', 'TU-2 Estimated Speed', 'TU-1 Speed', 'TU-2 

In [124]:
# Select columns for analysis 
analysis_columns = [
    'Crash Date', 
    'City', 
    'County',
    'Crash Year',
    'Location 1', 
    'Location 2',
    'Link',
    'Latitude', 
    'Longitude',
]

cdot_pdf_select_columns = cdot_pdf_withNans_dedup[analysis_columns]

# check NaN count by column 
for col in cdot_pdf_select_columns.columns:
    print(col, cdot_pdf_select_columns.loc[cdot_pdf_select_columns[col].isna()].shape[0])

Crash Date 0
City 56265
County 0
Crash Year 0
Location 1 0
Location 2 21
Link 0
Latitude 10226
Longitude 10226


In [125]:
cdot_pdf_select_columns.describe()

,Crash Date,Crash Year,Latitude,Longitude
count,202146,202146.000000,191920.000000,191920.000000
mean,2023-12-31 00:50:05.574189056,2023.494593,39.549164,-105.218258
min,2023-01-01 00:00:00,2023.000000,36.994398,-109.050640
25%,2023-07-06 00:00:00,2023.000000,39.346003,-105.102510
50%,2023-12-27 00:00:00,2023.000000,39.718840,-104.985160
75%,2024-07-01 00:00:00,2024.000000,39.885370,-104.822927
max,2024-12-31 00:00:00,2024.000000,41.002640,-102.047018
std,NaN,0.499972,0.698164,0.952878


In [126]:
# fill null values that screw up counting
cdot_pdf = cdot_pdf_select_columns.copy()
    
# Fill NaN values 
#cdot_pdf['Crash Date'] = cdot_pdf['Crash Date'].fillna('no crash date provided')
#cdot_pdf['Crash Year'] = cdot_pdf['Crash Year'].fillna('no crash date provided')
#cdot_pdf['City'] = cdot_pdf['City'].fillna('no city provided')
#cdot_pdf['County'] = cdot_pdf['County'].fillna('no county provided')
#cdot_pdf['Latitude'] = cdot_pdf['Latitude'].fillna(999.999000)
#cdot_pdf['Longitude'] = cdot_pdf['Longitude'].fillna(999.999000)

# set default Denver latitude and longitude value to 999.999000
default_lat = 39.74602
default_long = -104.98877
default_lat_long_condition = (cdot_pdf['Latitude'] == default_lat) & (cdot_pdf['Longitude'] == default_long)
cdot_pdf.loc[default_lat_long_condition, 'Latitude'] = 999.999000
cdot_pdf.loc[default_lat_long_condition, 'Longitude'] = 999.999000

# mark "unknown" 999.999 city, Location 1, Location 2 as null
cdot_pdf['City'] = cdot_pdf['City'].fillna('UNKNOWN')
cdot_pdf['Location 1'] = cdot_pdf['Location 1'].fillna('UNKNOWN')
cdot_pdf['Location 2'] = cdot_pdf['Location 1'].fillna('UNKNOWN')

cdot_pdf.loc[cdot_pdf['City'].str.contains('UNKNOWN'), 'City'] = None
cdot_pdf.loc[cdot_pdf['Location 1'].str.contains('UNKNOWN'), 'Location 1'] = None
cdot_pdf.loc[cdot_pdf['Location 2'].str.contains('UNKNOWN'), 'Location 2'] = None
cdot_pdf.loc[cdot_pdf['Latitude'] == 999.999000] = None 
cdot_pdf.loc[cdot_pdf['Longitude'] == 999.999000] = None 

In [127]:
# check NaN count by column 
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])

Crash Date 117
City 56379
County 117
Crash Year 117
Location 1 137
Location 2 137
Link 117
Latitude 10343
Longitude 10343


In [128]:
display(cdot_pdf.head(10))

,Crash Date,City,County,Crash Year,Location 1,Location 2,Link,Latitude,Longitude
0,2023-01-01,FORT COLLINS,LARIMER,2023.0,VERMONT DR,VERMONT DR,AT,40.543810,-105.037120
1,2023-01-01,FORT COLLINS,LARIMER,2023.0,E HARMONY RD,E HARMONY RD,AT,40.523370,-105.039350
2,2023-01-01,DENVER,DENVER,2023.0,S KALAMATH ST,S KALAMATH ST,00200FN,39.713250,-104.999410
3,2023-01-01,AURORA,ARAPAHOE,2023.0,E LINVALE AVE,E LINVALE AVE,00050FW,39.666990,-104.802350
4,2023-01-01,WESTMINSTER,JEFFERSON,2023.0,N SHERIDAN BLVD,N SHERIDAN BLVD,AT,39.841740,-105.053190
5,2023-01-01,THORNTON,ADAMS,2023.0,15500 I-25 NB,15500 I-25 NB,AT,39.975803,-104.988003
6,2023-01-01,PUEBLO,PUEBLO,2023.0,W 6TH ST,W 6TH ST,AT,38.272500,-104.608640
7,2023-01-01,PUEBLO,PUEBLO,2023.0,W NORTHERN AVE,W NORTHERN AVE,AT,38.244080,-104.643830
8,2023-01-01,DENVER,DENVER,2023.0,50 S FEDERAL BLVD,50 S FEDERAL BLVD,99999FS,39.715730,-105.025100
9,2023-01-01,None,DOUGLAS,2023.0,C-470,C-470,AT,39.564770,-105.037250


### Filter to El Paso County for simplicity 

In [129]:
cdot_pdf_elpaso = cdot_pdf.loc[cdot_pdf['County'] == 'EL PASO']
print(cdot_pdf_elpaso.shape)

# check NaN count by column 
for col in cdot_pdf_elpaso.columns:
    print(col, cdot_pdf_elpaso.loc[cdot_pdf_elpaso[col].isna()].shape[0])

(19856, 9)
Crash Date 0
City 5042
County 0
Crash Year 0
Location 1 3
Location 2 3
Link 0
Latitude 650
Longitude 650


### Inspect Missing Lat Long

In [130]:
cdot_elpaso_noLL = cdot_pdf_elpaso.loc[cdot_pdf_elpaso['Latitude'].isna() | cdot_pdf_elpaso['Longitude'].isna()]
display(cdot_elpaso_noLL)

,Crash Date,City,County,Crash Year,Location 1,Location 2,Link,Latitude,Longitude
34,2023-01-01,COLORADO SPRINGS,EL PASO,2023.0,E PLATTE AVE,E PLATTE AVE,AT,NaN,NaN
47,2023-01-01,COLORADO SPRINGS,EL PASO,2023.0,N CIRCLE DR,N CIRCLE DR,AT,NaN,NaN
64,2023-01-01,COLORADO SPRINGS,EL PASO,2023.0,N 25TH ST,N 25TH ST,AT,NaN,NaN
78,2023-01-01,COLORADO SPRINGS,EL PASO,2023.0,E FILLMORE ST,E FILLMORE ST,AT,NaN,NaN
93,2023-01-01,COLORADO SPRINGS,EL PASO,2023.0,SUMMERSET DR,SUMMERSET DR,AT,NaN,NaN
...,...,...,...,...,...,...,...,...,...
33352,2024-04-30,MANITOU SPRINGS,EL PASO,2024.0,RUXTON AVE,RUXTON AVE,WO,NaN,NaN
45183,2024-06-14,None,EL PASO,2024.0,I-25,I-25,AT,NaN,NaN
48039,2024-06-26,None,EL PASO,2024.0,HWY 24,HWY 24,AT,NaN,NaN
72836,2024-09-25,MANITOU SPRINGS,EL PASO,2024.0,RUXTON AVE,RUXTON AVE,00128FW,NaN,NaN


In [131]:
# Write out to csv 
#cdot_elpaso_noLL.to_csv('crashes_elpaso_w_no_latlong_2024.csv', index=False) 

#### Attempt to Fill Missing Lat Long

In [165]:
def get_address_from_loc1loc2city(cdot_pdf):

    # create an address from the Location 1, Location 2, and City columns 
    cdot_pdf.loc[:, 'address'] = cdot_elpaso_noLL.loc[:, 'Location 1'] + ' ' + \
                                         cdot_elpaso_noLL.loc[:, 'City'] + ' ' + \
                                        'CO'
    # check if 
    # Location 1 and Location 2 are identical 
    # AND there is no number either Location 1 or Location 2
    # Example: N MAIN ST, N MAIN ST
    exact_same_loc1loc2_no_numbers = (cdot_pdf['Location 1'] == cdot_pdf['Location 2']) & \
                (~cdot_pdf['Location 1'].str.contains(r'\d')) & \
                (~cdot_pdf['Location 2'].str.contains(r'\d'))
    cdot_pdf.loc[exact_same_loc1loc2_no_numbers, 'address'] = None 

    # check if 
    # Location 1 and Location 2 are identical 
    # AND contains HWY, I-##, XXST, XXTH 
    # Example: I-25, I-25
    # Example: 31ST ST, 31ST ST
    exact_same_loc1loc2_and_highway_or_numberstreet = (cdot_pdf['Location 1'] == cdot_pdf['Location 2']) & \
                (cdot_pdf['Location 1'].str.contains(' HWY ')) | \
                (cdot_pdf['Location 1'].str.contains(r'I-\d\d')) | \
                (cdot_pdf['Location 1'].str.contains(r'\sI\s\d\d')) | \
                (cdot_pdf['Location 1'].str.contains(r'\s\d{1,2}TH\s')) | \
                (cdot_pdf['Location 1'].str.contains(r'\s\d{1,2}ST\s'))
    cdot_pdf.loc[exact_same_loc1loc2_and_highway_or_numberstreet, 'address'] = None 
    
    return cdot_pdf

# had to drop nulls to get condition to work 
cdot_elpaso_noLL_TEMP = cdot_elpaso_noLL.dropna(subset=['Location 1', 'Location 2'])

cdot_elpaso_noLL_TEMP = get_address_from_loc1loc2city(cdot_elpaso_noLL_TEMP)
display(cdot_elpaso_noLL_TEMP[['City', 'Location 1', 'Location 2', 'address']])

/var/folders/wh/mh9pm3896vb556vt9fxjp6sc0000gq/T/ipykernel_10054/2539301831.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdot_pdf.loc[:, 'address'] = cdot_elpaso_noLL.loc[:, 'Location 1'] + ' ' + \


,City,Location 1,Location 2,address
34,COLORADO SPRINGS,E PLATTE AVE,E PLATTE AVE,None
47,COLORADO SPRINGS,N CIRCLE DR,N CIRCLE DR,None
64,COLORADO SPRINGS,N 25TH ST,N 25TH ST,None
78,COLORADO SPRINGS,E FILLMORE ST,E FILLMORE ST,None
93,COLORADO SPRINGS,SUMMERSET DR,SUMMERSET DR,None
...,...,...,...,...
33352,MANITOU SPRINGS,RUXTON AVE,RUXTON AVE,None
45183,None,I-25,I-25,None
48039,None,HWY 24,HWY 24,NaN
72836,MANITOU SPRINGS,RUXTON AVE,RUXTON AVE,None


In [166]:
from geopy.geocoders import Nominatim
import numpy as np

geolocator = Nominatim(user_agent="hmn")
def get_latlong_from_address(address):

    #print(address)
    if (address is not None) & (address != np.nan) & (type(address) == str):
        try:
            location = geolocator.geocode(address)
                
            latitude = location.latitude
            longitude = location.longitude
            #print(f"Latitude: {latitude}, Longitude: {longitude}")
            #print()
            return (latitude, longitude)
        except:
            print(f"Could not find coordinates for {address}")
            print()
            return None

    return None

In [167]:
cdot_elpaso_noLL_TEMP['LatLong'] = cdot_elpaso_noLL_TEMP['address'].apply(get_latlong_from_address)

display(cdot_elpaso_noLL_TEMP)

Could not find coordinates for 1100 N CIRCLE DR COLORADO SPRINGS CO

Could not find coordinates for I 25 NB ON RAMP COLORADO SPRINGS CO

Could not find coordinates for 1402 HOWARD AVE COLORADO SPRINGS CO

Could not find coordinates for 17TH ST COLORADO SPRINGS CO



/var/folders/wh/mh9pm3896vb556vt9fxjp6sc0000gq/T/ipykernel_10054/888478936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdot_elpaso_noLL_TEMP['LatLong'] = cdot_elpaso_noLL_TEMP['address'].apply(get_latlong_from_address)


,Crash Date,City,County,Crash Year,Location 1,Location 2,Link,Latitude,Longitude,address,LatLong
34,2023-01-01,COLORADO SPRINGS,EL PASO,2023.0,E PLATTE AVE,E PLATTE AVE,AT,NaN,NaN,None,None
47,2023-01-01,COLORADO SPRINGS,EL PASO,2023.0,N CIRCLE DR,N CIRCLE DR,AT,NaN,NaN,None,None
64,2023-01-01,COLORADO SPRINGS,EL PASO,2023.0,N 25TH ST,N 25TH ST,AT,NaN,NaN,None,None
78,2023-01-01,COLORADO SPRINGS,EL PASO,2023.0,E FILLMORE ST,E FILLMORE ST,AT,NaN,NaN,None,None
93,2023-01-01,COLORADO SPRINGS,EL PASO,2023.0,SUMMERSET DR,SUMMERSET DR,AT,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...
33352,2024-04-30,MANITOU SPRINGS,EL PASO,2024.0,RUXTON AVE,RUXTON AVE,WO,NaN,NaN,None,None
45183,2024-06-14,None,EL PASO,2024.0,I-25,I-25,AT,NaN,NaN,None,None
48039,2024-06-26,None,EL PASO,2024.0,HWY 24,HWY 24,AT,NaN,NaN,NaN,None
72836,2024-09-25,MANITOU SPRINGS,EL PASO,2024.0,RUXTON AVE,RUXTON AVE,00128FW,NaN,NaN,None,None


In [168]:
cdot_elpaso_noLL_TEMP.to_csv('temp.csv', index=False) 